## 初期設定

### ライブラリのimport

In [1]:
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator
from miditok.utils import split_files_for_training
from torch.utils.data import DataLoader
from pathlib import Path

from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import time
import icecream as ic
import os
from symusic import Score
import datetime

import json
import platform

/Users/hapticslab/Programming/humusic/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.empty_cache()

### deviceの設定

In [13]:
#device setting

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
torch.cuda.is_available()

False

## データセットとトークナイザーの設定

## 共通環境

In [4]:
platform_name = platform.system()
dataset_name = "piano1_only"
print(platform_name, dataset_name)

Darwin piano1_only


### tokenizrの設定

In [5]:
# Creating a multitrack tokenizer, read the doc to explore all the parameters
config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(config)

/Users/hapticslab/Programming/humusic/.venv/lib/python3.9/site-packages/miditok/tokenizations/remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


In [6]:
# JSONファイルを読み込む
with open("datasets_path.json", "r", encoding="utf-8") as json_file:
    loaded_paths = json.load(json_file)[platform_name][dataset_name]


# パスを取得
dataset_dir = Path(loaded_paths["dataset_dir"])
files_paths = list(dataset_dir.glob("**/*.mid"))
tokenizer_path = Path(loaded_paths["tokenizer_path"])
chunks_dir = Path(loaded_paths["chunks_dir"])
#directory_path = Path(loaded_paths["directory_path"])

print(dataset_dir, "\n", tokenizer_path, "\n", chunks_dir)
#print(f"Directory path: {directory_path}")

/Users/hapticslab/Programming/humusic/datasets/piano1_only 
 /Users/hapticslab/Programming/humusic/midi_generator/tokens/piano1_token/tokenizer.json 
 /Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only


### Tokenizerの作成とデータセットの分割（新規データのみ）

In [7]:

tokenizer.train(vocab_size=30000, files_paths=files_paths)

tokenizer.save(tokenizer_path)

split_files_for_training(
    files_paths=files_paths,
    tokenizer=tokenizer,
    save_dir=chunks_dir,
    max_seq_len=1024,
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Splitting music files (/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only): 100%|██████████| 280/280 [00:00<00:00, 1456.85it/s]


[PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_muss_2_0.mid'),
 PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_grieg_butterfly_0.mid'),
 PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_burg_erwachen_0.mid'),
 PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_alb_se8_0.mid'),
 PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_alb_se8_1.mid'),
 PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_mendel_op62_3_0.mid'),
 PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_haydn_35_3_0.mid'),
 PosixPath('/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only/piano1_only_haydn_35_2_0.mid'),
 PosixPath('/Users/hapticslab/Programmi

In [8]:

tokenizer = REMI(params=Path("tokens/piano1_token/tokenizer.json"))

# Create a Dataset, a DataLoader and a collator to train a model
dataset = DatasetMIDI(
    files_paths=list(chunks_dir.glob("**/*.mid")),
    tokenizer=tokenizer,
    max_seq_len=1024,
)
collator = DataCollator(tokenizer.pad_token_id, copy_inputs_as_labels=True)
dataloader = DataLoader(dataset, batch_size=16, collate_fn=collator)

/Users/hapticslab/Programming/humusic/.venv/lib/python3.9/site-packages/miditok/tokenizations/remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


### Mac環境

In [4]:

# Train the tokenizer with Byte Pair Encoding (BPE)
#files_paths = list(Path("../datasets").glob("**/*.mid"))
dataset_dir = Path("/Users/hapticslab/Programming/humusic/datasets/orchestra")
#dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/orchestra")
#dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/piano1_only")
files_paths = list(dataset_dir.glob("**/*.mid"))

tokenizer.train(vocab_size=30000, files_paths=files_paths)

#tokenizer.save(Path("path", "to", "save", "tokenizer.json"))
'''
tokenizer.save(Path("/Users/hapticslab/Programming/humusic/midi_generator/tokens/orchestra_token/tokenizer.json"))
'''
#tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/orchestra_token/tokenizer.json"))
#tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/piano1_token/tokenizer.json"))
# And pushing it to the Hugging Face hub (you can download it back with .from_pretrained)
#tokenizer.push_to_hub("username/model-name", private=True, token="your_hf_token")

# Split MIDIs into smaller chunks for training
dataset_chunks_dir = Path("/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/orchestra")
#dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/orchestra")
#dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/piano1_only")



### Windows環境

In [5]:



# Train the tokenizer with Byte Pair Encoding (BPE)
#files_paths = list(Path("../datasets").glob("**/*.mid"))
#dataset_dir = Path("/Users/hapticslab/Programming/humusic/datasets/piano1_only")
#dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/orchestra")
dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/piano1_only")
#dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/maestro-v3.0.0")
files_paths = list(dataset_dir.glob("**/*.mid"))


#tokenizer.train(vocab_size=30000, files_paths=files_paths)


#tokenizer.save(Path("path", "to", "save", "tokenizer.json"))

#tokenizer.save(Path("/Users/hapticslab/Programming/humusic/midi_generator/tokens/piano1_token/tokenizer.json"))

tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/piano1_token/tokenizer.json"))
#tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/maestro/tokenizer.json"))
# And pushing it to the Hugging Face hub (you can download it back with .from_pretrained)
#tokenizer.push_to_hub("username/model-name", private=True, token="your_hf_token")

# Split MIDIs into smaller chunks for training
#dataset_chunks_dir = Path("/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only")
dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/piano1_only")
#dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/maestro")


In [4]:
tokenizer = REMI(params=Path("tokens/piano1_token/tokenizer.json"))

C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\miditok\tokenizations\remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


### トークンをDataloaderに保存

In [25]:
'''
split_files_for_training(
    files_paths=files_paths,
    tokenizer=tokenizer,
    save_dir=dataset_chunks_dir,
    max_seq_len=1024,
)
'''


# Create a Dataset, a DataLoader and a collator to train a model
dataset = DatasetMIDI(
    files_paths=list(dataset_chunks_dir.glob("**/*.mid")),
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
collator = DataCollator(tokenizer.pad_token_id, copy_inputs_as_labels=True)
dataloader = DataLoader(dataset, batch_size=64, collate_fn=collator)

## Attention付きLSTMによる学習，生成

学習パラメータの設定

In [9]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# LSMT with Attention
from LSTMwithAtt import LSTMwithAtt


hidden_size = 200
model = LSTMwithAtt(tokenizer, hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

num_epoch = 100

model = model.to(device)


In [ ]:
model.train()
start_time = time.time()

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    i = 0
    loss  = 0
    

    for batch in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epoch} |', unit='batch'):
        encoder_input = batch["input_ids"][:,1:].to(device)
        decoder_input = batch["input_ids"][:,:-1].to(device)
        labels = batch["input_ids"][:,1:].to(device)
        labels_one_hot = F.one_hot(labels, num_classes=tokenizer.vocab_size).float()
        
        out = model(encoder_input, decoder_input)
        loss = criterion(out[0],labels_one_hot[0])
        
        for h in range(1,len(out)):
            loss += criterion(out[h],labels_one_hot[h])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
    
    print(f'epoch {epoch+1}/{num_epoch} | loss {loss} | time {time.time()-epoch_start_time}')
    outfile = "models/" + "lstmwithatt_" + str(hidden_size) + "_" + dataset_name + "_" + str(epoch+1) + ".pt"
    torch.save(model.state_dict(),outfile)  
    

Epoch 1/100 |: 100%|██████████| 8/8 [00:16<00:00,  2.09s/batch]


epoch 1/100 | loss 53.097999572753906 | time 16.68554973602295


Epoch 2/100 |: 100%|██████████| 8/8 [00:14<00:00,  1.87s/batch]


epoch 2/100 | loss 35.28253936767578 | time 14.983150005340576


Epoch 3/100 |: 100%|██████████| 8/8 [00:14<00:00,  1.80s/batch]


...

Epoch 42/100 |: 100%|██████████| 8/8 [00:16<00:00,  2.05s/batch]


epoch 42/100 | loss 14.912382125854492 | time 16.414351224899292


Epoch 43/100 |: 100%|██████████| 8/8 [00:15<00:00,  1.96s/batch]


epoch 43/100 | loss 14.841286659240723 | time 15.671698093414307


Epoch 44/100 |:  38%|███▊      | 3/8 [00:05<00:09,  1.94s/batch]

In [27]:
#masked input

model.train()
start_time = time.time()

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    i = 0
    loss  = 0
    

    for batch in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epoch} |', unit='batch'):
        inputs_shape = batch["input_ids"].shape  # トークンID列    
        mask_border_index = inputs_shape[1]//4
        masked_inputs = batch["input_ids"][:,1:].clone().to(device)
        masked_inputs[:,mask_border_index:] = tokenizer.pad_token_id  # マスクされた入力
        
        #encoder_input = batch["input_ids"][:,1:].to(device)
        decoder_input = batch["input_ids"][:,:-1].to(device)
        labels = batch["input_ids"][:,1:].to(device)
        labels_one_hot = F.one_hot(labels, num_classes=tokenizer.vocab_size).float()
        
        out = model(masked_inputs, decoder_input)
        loss = criterion(out[0],labels_one_hot[0])
        
        for h in range(1,len(out)):
            loss += criterion(out[h],labels_one_hot[h])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
    
    print(f'epoch {epoch+1}/{num_epoch} | loss {loss} | time {time.time()-epoch_start_time}')
    outfile = "models/" + "lstmwithatt_" + "maskedinput_" + str(hidden_size) + "_" + dataset_name + "_" + str(epoch+1) + ".pt"
    torch.save(model.state_dict(),outfile)  
    

Epoch 1/100 |: 100%|██████████| 8/8 [00:19<00:00,  2.39s/batch]


epoch 1/100 | loss 71.67308807373047 | time 19.12471103668213


Epoch 2/100 |: 100%|██████████| 8/8 [00:18<00:00,  2.37s/batch]


epoch 2/100 | loss 70.7986068725586 | time 18.9780433177948


Epoch 3/100 |: 100%|██████████| 8/8 [00:18<00:00,  2.35s/batch]


...

epoch 98/100 | loss 18.83961296081543 | time 18.11105489730835


Epoch 99/100 |: 100%|██████████| 8/8 [00:18<00:00,  2.27s/batch]


epoch 99/100 | loss 18.714792251586914 | time 18.197400093078613


Epoch 100/100 |: 100%|██████████| 8/8 [00:17<00:00,  2.25s/batch]

epoch 100/100 | loss 18.58608627319336 | time 17.968130826950073


### 生成-プロンプトのトークン化

In [11]:
# 入力トークン列の例
prompt = ("prompt/piano1_only_bach_850.mid")

input_ids = torch.tensor([tokenizer(prompt)]).to(device)
input_ids.shape, input_ids

(torch.Size([1, 490]),
 tensor([[ 2973,  3465,    58,  8344,  2729, 13485,  3300, 11913,  1503,  2650,
          18858,  4918,  3197, 20006,  3091,  1458,  6462,  3850,  2259, 22554,
           2650,  1845,  2873,  1235,   482,  2436,  3465,    58,  8344,  4641,
           5528,  1501,  5040, 11913,  1503,  5266,  2024,  4303,  1559,  8208,
           3197, 11135,  2731,  3091,  1458,  6462,  2778,  3194,  2290,  1229,
           7308, 18991,  3786, 22734,  1502,  9196,  2108,  6167,  4255,  6899,
           8341,  3827, 22968,  3911,  2048,  5039,  2254,  7291, 23365,  5517,
           2211, 18528,  7858, 18145,  1097, 11482, 22734,  3487,    46,  4187,
           6897,  4255, 10990,  4428,  5019,  2018,    41,   382, 22968,   369,
          15713, 17062,    41,   365, 18461, 15636,   378,  8341,  7308, 28398,
           3081, 26001,    48,   339,  6526,  4104,  5514,  8386,  4668,  1906,
           4464,  3406,  6543,  5893,  2561,  1582,  3367,  9282,  4474,  2118,
           4111, 

生成

In [17]:
# Generate a MIDI file

model.load_state_dict(torch.load("lstmwithatt_better.model",map_location=torch.device('cpu')))


#torch.set_printoptions(threshold=torch.inf)
model.eval()

gen_token = []

with torch.no_grad():
    #prompt_input = torch.LongTensor( input_ids ).to(device)
    #print(input_ids.shape)
    x = model.input_emb(input_ids)
    #print(x.shape)
    ox, (hnx, cnx) = model.lstm1(x)
    #print(ox.shape, hnx.shape,cnx.shape)
    hnx, cnx = hnx[:,0,:], cnx[:,0,:]
    
    wid = input_ids[0][0]
    
    sl = 0
    while True:
        wids = torch.LongTensor([wid]).to(device)
        y = model.answer_emb(wids)
        #print(y.shape)
        
        oy, (hnx, cnx) = model.lstm2(y, (hnx, cnx))
        oy = oy.unsqueeze(1)
        ox1 = ox.permute(0,2,1)
        sim = torch.bmm(oy,ox1)
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws,xws)
        alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
        ct = torch.bmm(alpha,ox)
        oy1 = torch.cat([ct,oy],dim=2)
        oy2 = model.Wc(oy1)
        oy3 = model.W(oy2)
        wid = torch.argmax(oy3[0]).item()
        gen_token.append(wid)
        #if (wid == esid):
            #break
        if (sl == 2000):
            break
        sl += 1
    print(gen_token)

/var/folders/7f/fjwjx3xd5mbf2t9x4r4k4__00000gn/T/ipykernel_21876/1712198453.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lstmwithatt

[6877, 17955, 1735, 19508, 11337, 29262, 6373, 1889, 8018, 6877, 1889, 15789, 7996, 8825, 23004, 9739, 9848, 7027, 9896, 1660, 5685, 2781, 17888, 4171, 11835, 5685, 16650, 2781, 17888, 555, 11337, 7674, 12799, 13156, 17888, 3363, 11337, 15210, 17888, 17888, 17888, 23004, 5718, 17497, 17888, 4879, 8185, 20484, 25, 10102, 19377, 19508, 4879, 1889, 12032, 885, 14379, 258, 22701, 17888, 17888, 2351, 11337, 18558, 4697, 4989, 12558, 10102, 15789, 15640, 6023, 15210, 17888, 17888, 16654, 909, 4582, 4879, 1889, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 

In [16]:
generated = tokenizer.decode(gen_token)
print("Generated MIDI Tokens:", generated)
generated_time = str(time.time())

generated.dump_midi("generated_test_"+ generated_time +".mid")
print("generated_test_"+ generated_time +".mid")

Generated MIDI Tokens: Score(ttype=Tick, tpq=8, begin=0, end=3170, tracks=1, notes=1848, time_sig=1, key_sig=0, markers=0)
generated_test_1734311796.2703722.mid


## Trasformerによる学習，生成

### 学習パラメータの設定

In [10]:
from transformers import GPT2Config, GPT2LMHeadModel


config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=1024,
    n_ctx=1024,
    n_embd=256,
    n_layer=4,
    n_head=4,
    
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    pad_token_id=tokenizer.pad_token_id
    
    )

model = GPT2LMHeadModel(config).to(device)

# 損失関数とオプティマイザ
criterion = torch.nn.CrossEntropyLoss()  # トークンの予測タスクに使う
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # 適切な学習率を設定

### 学習

In [27]:
torch.set_printoptions(edgeitems=torch.inf)

# 学習ループ
epochs = 300 # エポック数

for epoch in range(epochs):
    model.train()  # モデルを学習モードに切り替え
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs_shape = batch["input_ids"].shape  # トークンID列    
        
        
        mask_border_index = inputs_shape[1]//4
        masked_inputs = batch["input_ids"][:,1:].clone().to(device)
        masked_inputs[:,mask_border_index:] = tokenizer.pad_token_id  # マスクされた入力
        
        
        labels = batch["labels"][:,:-1].to(device)    # 正解ラベル
        attention_mask = batch["attention_mask"].to(device)  # マスク（任意）
        
        
        
        #print(inputs[0],labels[0])
        
        # 勾配を初期化
        optimizer.zero_grad()

        # モデルの前方計算
        outputs = model(
            input_ids=inputs,
            labels=labels,
            attention_mask=attention_mask,
        )
        loss = outputs.loss  # GPT2LMHeadModelは自動で損失を計算する

        # 勾配の計算とパラメータの更新
        loss.backward()
        optimizer.step()
        
        # 損失を記録
        total_loss += loss.item()
        

    # エポックごとの平均損失を出力
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    
    #model.save_pretrained("models/transformer/" + "piano1_only/test" + str(epoch+1) + ".pt")

Epoch 1/300: 100%|██████████| 29/29 [00:15<00:00,  1.85it/s]


Epoch 1/300 - Loss: 4.4964


Epoch 2/300: 100%|██████████| 29/29 [00:15<00:00,  1.86it/s]


Epoch 2/300 - Loss: 4.4984


Epoch 3/300: 100%|██████████| 29/29 [00:15<00:00,  1.89it/s]


Epoch 3/300 - Loss: 4.4974


Epoch 4/300:  24%|██▍       | 7/29 [00:03<00:12,  1.76it/s]


KeyboardInterrupt: 

: 

In [11]:
model.save_pretrained("models/transformer/" + "piano1_only/shifted_test" + str(epoch+1) + ".pt")

NameError: name 'epoch' is not defined

### 生成-プロンプトのトークン化

In [12]:


# 入力トークン列の例
prompt = ("prompt/output.mid")

input_ids = torch.tensor([tokenizer(prompt)]).to(device)
input_ids

tensor([[  4, 175, 280,   9, 102, 109, 176, 280,  32, 102, 114, 182, 280,  28,
         102, 109, 183, 280,  33, 102, 110, 185, 280,  32, 102, 112, 189, 280,
          28, 102, 109, 191, 280,  35, 102, 109, 192, 280,  40, 102, 110, 194,
         280,  39, 102, 112, 198, 280,  38, 102, 116,   4, 175, 280,  29, 102,
         109, 176, 280,  41, 102, 114, 182, 280,  36, 102, 109, 183, 280,  17,
         102, 109, 184, 280,  41, 102, 115, 191, 280,  36, 102, 109, 192, 280,
          39, 102, 117]], device='cuda:0')

In [ ]:
#ぱっでぃんぐは不要でした

# 1次元で50要素を持つベクトル
vector = input_ids

# 1024要素に拡張するためのパディング
target_length = 1023
padding_length = target_length - input_ids.size(1)

# 末尾にパディングを追加
padded_vector = F.pad(input_ids, (0, padding_length), value=0)  # value=0 はパディングの値

print(padded_vector)
print(padded_vector.size())  # torch.Size([1024])

tensor([[  4, 175, 280,   9, 102, 109, 176, 280,  32, 102, 114, 182, 280,  28,
         102, 109, 183, 280,  33, 102, 110, 185, 280,  32, 102, 112, 189, 280,
          28, 102, 109, 191, 280,  35, 102, 109, 192, 280,  40, 102, 110, 194,
         280,  39, 102, 112, 198, 280,  38, 102, 116,   4, 175, 280,  29, 102,
         109, 176, 280,  41, 102, 114, 182, 280,  36, 102, 109, 183, 280,  17,
         102, 109, 184, 280,  41, 102, 115, 191, 280,  36, 102, 109, 192, 280,
          39, 102, 117,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0

### 生成-メインプロセス

In [14]:
#regenerate

# モデルの読み込み
model = GPT2LMHeadModel.from_pretrained("models/transformer/piano1_only/shifted_test27.pt")
#model = GPT2LMHeadModel.from_pretrained("models/transformer/" + "piano1_only/shifted_test" + "26" + ".pt")
#C:\Users\keisu\Programming\humor\humusic\midi_generator\models\transformer\piano1_only\shifted_test27.pt
model.to(device)
model.eval()

# モデルによる生成
# output = model.generate(
#     input_ids=input_ids,
#     max_length=1024,
#     num_beams=5,
#     no_repeat_ngram_size=2,
# )

# モデルによる生成
with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        max_length=1024,
        num_beams=5,  # ビームサーチ
        no_repeat_ngram_size=2,  # 同じn-gramを繰り返さない
        temperature=1.2,  # サンプリングの多様性を調整
        top_k=50,  # 上位k個のトークンをサンプリング
        top_p=0.95,  # 累積確率cutoff
        )

C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


### 生成-結果をdetokenize

In [15]:
# トークン列をデコードして結果を表示
generated = tokenizer.decode(output[0].tolist())
print("Generated MIDI Tokens:", generated)

generated.dump_midi("maskedinput_transformer_shifted_10_5_pad.mid")

Generated MIDI Tokens: Score(ttype=Tick, tpq=8, begin=0, end=1368, tracks=1, notes=49, time_sig=1, key_sig=0, markers=0)


## python(GPU)のキャッシュクリア

In [ ]:
del model, dataset, dataloader, collator
torch.cuda.empty_cache()

## 生成したmidiの再生
強制終了するとipykernelがクラッシュするので注意

In [76]:
#pygameによる再生

file_name = Path("generated_test_"+ generated_time +".mid")

import time
import pygame

# Initialize pygame
pygame.init()

# Set up the mixer
pygame.mixer.init()

# Load the MIDI file
#pygame.mixer.music.load('bach_850.mid')
pygame.mixer.music.load(file_name)
#pygame.mixer.music.load('test.mid')

# Play the MIDI file
pygame.mixer.music.set_volume(0.25)
pygame.mixer.music.play()


# Keep the program running until the music stops
while pygame.mixer.music.get_busy():
    if input("Press 'q' to quit: ") == 'q':
        break
    pygame.time.Clock().tick(10)

# Quit pygame
pygame.quit()

# prompt.midの作成セル

In [29]:
# まずは1小節のnote（ド、C4）を1つ生成してみる
import mido
from mido import Message, MidiFile, MidiTrack, MetaMessage

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(120))) # bpm120

track.append(Message('note_on', note=74, velocity=57, time=120))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_on', note=78, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=69, velocity=60, time=100))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_off', note=69, velocity=0, time=100))
track.append(Message('note_on', note=78, velocity=57, time=0))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_on', note=74, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=81, velocity=62, time=100))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_off', note=81, velocity=0, time=100))
track.append(Message('note_on', note=74, velocity=58, time=0))
track.append(Message('note_on', note=76, velocity=58, time=120))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_on', note=78, velocity=58, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=69, velocity=61, time=100))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_off', note=69, velocity=0, time=100))
track.append(Message('note_on', note=78, velocity=57, time=0))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_on', note=74, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=79, velocity=60, time=100))
track.append(Message('note_off', note=74, velocity=0, time=20))


mid.save('prompt.mid') # MidiFileを保存
